###############################################################################
# SrswtiRanker
###############################################################################

In [ ]:
# ---------------------------  Imports -------------------------------
from srswti_axis import SrswtiRanker
from utils import call_groq_llm, SAMPLE_DOCS
# ---------------------------  Basic 3 Demos -------------------------
basic_ranker = SrswtiRanker()
ranker_docs = [
    "PyTorch is a deep learning framework",
    "TensorFlow is for large-scale machine learning",
    "Keras builds on top of TensorFlow",
]

/home/zhreyas/miniconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-10 15:15:09.699301: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741599909.718948  944293 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741599909.724249  944293 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 15:15:09.745731: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-c

In [ ]:
# Use the basic_ranker to rank the documents based on their relevance to the query
# The rank_documents method returns documents sorted by relevance score (highest first)
ranking_res_1 = basic_ranker.rank_documents("machine learning frameworks", ranker_docs)
print("== Ranker 1 ==")
print(ranking_res_1, "\n")  # Print the ranked documents with newline for readability

== Ranker 1 ==
[('TensorFlow is for large-scale machine learning', 0.47151875495910645), ('PyTorch is a deep learning framework', 0.28279903531074524), ('Keras builds on top of TensorFlow', 0.1134694516658783)] 



In [ ]:
# Use the filter_documents method to select only documents that meet a threshold relevance score


filtered_docs_2 = basic_ranker.filter_documents("TensorFlow library", ranker_docs, threshold=0.6)
print("== Ranker 2 ==")
print(filtered_docs_2, "\n")  # Prints only documents with relevance score >= 0.6

== Ranker 2 ==
['TensorFlow is for large-scale machine learning', 'Keras builds on top of TensorFlow'] 



In [5]:

top_k_3 = basic_ranker.get_top_k("deep learning", ranker_docs, k=2)
print("== Ranker 3 ==")
print(top_k_3, "\n")

== Ranker 3 ==
['PyTorch is a deep learning framework', 'TensorFlow is for large-scale machine learning'] 



In [ ]:
# -------------------  RAG Demos ----------------------
def rag_demo_a(user_query, docs=SAMPLE_DOCS):
    """
    Demonstrates a simple RAG system that selects the top-ranked document 
    and passes it to the LLM to generate a response.
    
    Args:
        user_query (str): The user's query or question
        docs (list): List of documents to search through (defaults to SAMPLE_DOCS)
        
    Returns:
        None: Prints LLM response to console
    """
    ranked = basic_ranker.rank_documents(user_query, docs)
    if ranked:
        top_doc = ranked[0]
    else:
        top_doc = "No doc"
    prompt = f"User query: {user_query}\nTop doc: {top_doc}\nRespond to the query."
    resp = call_groq_llm(prompt)
    print("[Theorem9 RAG Demo A] LLM:\n", resp, "\n")

def rag_demo_b(user_query, docs=SAMPLE_DOCS):
    """
    Demonstrates a more advanced RAG system that filters documents based on
    relevance threshold, combines all relevant documents, and asks the LLM
    to summarize the information.
    
    Args:
        user_query (str): The user's query or question
        docs (list): List of documents to search through (defaults to SAMPLE_DOCS)
        
    Returns:
        None: Prints LLM response to console
    """
    relevant = basic_ranker.filter_documents(user_query, docs, threshold=0.2)
    combined = "\n".join(relevant) if relevant else "No relevant docs."
    prompt = f"User query: {user_query}\nRelevant docs:\n{combined}\nSummarize."
    resp = call_groq_llm(prompt)
    print("[Theorem9 RAG Demo B] LLM:\n", resp, "\n")


In [ ]:
rag_demo_a("Tell me about classical ML vs neural networks", SAMPLE_DOCS)


[Theorem9 RAG Demo A] LLM:
 [Groq LLM] When comparing classical machine learning (ML) to neural networks, there are significant differences in their approach, capabilities, and applications. 

Classical ML typically involves traditional algorithms like decision trees, linear regression, and support vector machines. These methods are often used for simpler problems and are more interpretable, meaning it's easier to understand how they arrive at their predictions. However, they can be limited in their ability to handle complex, high-dimensional data.

On the other hand, neural networks are computational models inspired by the human brain's architecture. They consist of interconnected nodes (neurons) organized in layers: input, hidden, and output layers. Neural networks are capable of learning complex patterns in data, especially in cases where the data is high-dimensional, like images or speech. They have been particularly successful in areas such as image and speech recognition, natural

In [ ]:
rag_demo_b("quantum computing for cryptography", SAMPLE_DOCS)


[Theorem9 RAG Demo B] LLM:
 [Groq LLM] Quantum computing has the potential to significantly impact cryptography, as it can perform certain calculations exponentially faster than classical computers. This could break widely-used encryption schemes that rely on factoring large prime numbers, using algorithms like Shor's. As a result, post-quantum cryptography methods are being developed to withstand quantum attacks. However, practical quantum computers capable of breaking current encryption are still years away due to challenges like qubit stability, error correction, and scaling issues. Companies are working to build more powerful quantum processors, but building fault-tolerant, general-purpose quantum computers remains an ongoing challenge. 

